In [1]:
import pandas as pd
import numpy as np

# join two csv (grid and aq and ob)

## concat grid weather data from 201701 to 201804

In [15]:
grid_17_18 = pd.read_csv("./data/MSBD5002PROJECT_data/gridWeather_201701-201803.csv")
grid_04 = pd.read_csv("./data/MSBD5002PROJECT_data/gridWeather_201804.csv")

In [19]:
# drop different columns
grid_04 = grid_04.drop(["weather","id"],axis=1)

In [21]:
# drop different columns
grid_17_18["time"] = grid_17_18["utc_time"]
grid_17_18["station_id"] = grid_17_18["stationName"]
grid_17_18["wind_speed"] = grid_17_18["wind_speed/kph"]
grid_17_18 = grid_17_18.drop(["utc_time","stationName","longitude","latitude","wind_speed/kph"],axis=1)

In [23]:
grid_17_18.head(1)

,temperature,pressure,humidity,wind_direction,time,station_id,wind_speed
0,-5.47,984.73,76.6,53.71,2017-01-01 00:00:00,beijing_grid_000,3.53


In [24]:
grid_04.head(1)

,station_id,time,temperature,pressure,humidity,wind_direction,wind_speed
0,beijing_grid_000,2018-04-01 00:00:00,15.0,974.1536,21.0,346.93,4.18


In [25]:
grid_1701_1804 = pd.concat([grid_17_18,grid_04],ignore_index=True)

D:\Anaconda3\envs\stats\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


In [30]:
pd.DataFrame.to_csv(grid_1701_1804,"./data/MSBD5002PROJECT_data/grid_weather_1701_1804_unfill.csv",index=None)

## concat observed weather data from 201701 to 201804

In [2]:
ow_17_18 = pd.read_csv("./data/MSBD5002PROJECT_data/observedWeather_201701-201801.csv")
ow_2_3 = pd.read_csv("./data/MSBD5002PROJECT_data/observedWeather_201802-201803.csv")
ow_4 = pd.read_csv("./data/MSBD5002PROJECT_data/observedWeather_201804.csv")

In [3]:
ow_17_18.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 158047 entries, 0 to 158046
Data columns (total 10 columns):
station_id        158047 non-null object
longitude         158047 non-null float64
latitude          158047 non-null float64
utc_time          158047 non-null object
temperature       158047 non-null float64
pressure          158047 non-null float64
humidity          158047 non-null int64
wind_direction    157813 non-null float64
wind_speed        157813 non-null float64
weather           158047 non-null object
dtypes: float64(6), int64(1), object(3)
memory usage: 12.1+ MB


In [ ]:
ow_17_18["time"] = ow_17_18["utc_time"]
ow_17_18["station_id"] = ow_17_18["stationName"]
ow_17_18 = ow_17_18.drop(["utc_time","stationName","longitude","latitude"],axis=1)

In [ ]:
ow_17_18[]

In [4]:
ow_2_3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26676 entries, 0 to 26675
Data columns (total 8 columns):
station_id        26676 non-null object
utc_time          26676 non-null object
weather           26676 non-null object
temperature       26676 non-null float64
pressure          26676 non-null float64
humidity          26676 non-null int64
wind_speed        26676 non-null float64
wind_direction    26676 non-null int64
dtypes: float64(3), int64(2), object(3)
memory usage: 1.6+ MB


In [5]:
ow_4.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12544 entries, 0 to 12543
Data columns (total 9 columns):
id                12544 non-null int64
station_id        12544 non-null object
time              12544 non-null object
weather           12544 non-null object
temperature       12544 non-null float64
pressure          12544 non-null float64
humidity          12544 non-null float64
wind_speed        12544 non-null float64
wind_direction    12544 non-null float64
dtypes: float64(5), int64(1), object(3)
memory usage: 882.1+ KB


### construct last hour, last 24hours feature for each row

### function fill na value in air quality  

In [13]:
path = ""
air_quality_df = pd.read_csv(path, index_col=None)

def traverse_head_tail(aq_df):
    aq_cols = aq_df.columns
    l = aq_df.shape[0]
    wd_size = 5
    
    for each_col in aq_cols:
        col = aq_df[each_col]
        head = 1
        tail = l-2
        
        # make sure these 2 are not na
        not_na_top = col.iloc[0]
        not_na_bot = col.iloc[l-1]
        
        while(tail >= head):
            # this row is na
            if col.iloc[head].isna():
                curr = head               
                for i in range(head,(head+wd_size)):
                    if not col.iloc[i].isna():
                        # record newest not na value
                        not_na_top = col.iloc[i]
                        # record 
                        head = i                       
                        break
                else:
                    head = head + wd_size
                    
                
                # rows above not null
                if not col.iloc[head-1].isna():
                    col.iloc[head] = np.mean([col.iloc[head-1],col.iloc[head+1]])
                         
            head += 1
            tail -= 1        

FileNotFoundError: File b'' does not exist

In [8]:
import pandas as pd
import numpy as np
from datetime import datetime
info_df = pd.read_csv("./data/lose_hour_record.csv")
aq_df = pd.read_csv("./data/air_quality_concated.csv")

In [5]:
# info_df[(info_df["pollutant"] == "PM2.5") & (info_df["counts"] >= 3000)].index.tolist()
# info_df[info_df["counts"] == 3][1000:1002]
# info_df.iloc[0].index
fill_threshold = 6
info_df.counts.unique()

array([   1,    4,    7,    2,   12,   17,   10,    5,    3,    6,   16,
          9,    8,   18,  246,   38,  168,   44,   25,   13,   11,   91,
         35,   22,   46,   14,   64,   20,   21,   15,   94,   32,   60,
         29,   19,   27,  165,  613,   37,   42,   31,   24,   23,  116,
         40,  167,   92,   34,   28,  247,   33,   81,   30,   50,   52,
         58,   45,   55,  614,   43,   26,  236,   74,   59,  172,   48,
         68,   53,   83,   80,   90,   51,   89,   84,   39,   97,   41,
         57,   70,   54,   36,  199,  162,   66,   61,   76,   65,   49,
        125,   71, 1981,  102,  114,  277,   88,   56,  122,  103,   77,
         47,  120,   72,  248,  169,   73,  772,  249, 2148,  227,  117,
         86,  251, 2278,  233,   69,  163,  176,   85,  268,   79,  108,
        255,  171,   95,  153, 1723, 1824, 1823,  144,   75,  189,   62,
         67,  174,  175,   63,  123,  878,  100,   93,  211,  232,  145,
        441,   87,  111,  478,  192,  808,  272,   

In [57]:
# info_df:['stationId', 'pollutant', 'start_time', 'counts']
# aq_df:stationId time PM2.5  PM10 NO2 CO O3 SO2 

new_info_df = info_df[info_df["counts"] <= fill_threshold]
print(new_info_df.shape[0])
pollutants = ["PM2.5", "PM10", "NO2", "CO", "O3" "SO2"]

d_format = "%Y-%m-%d %H:%M:%S"
# first filter counts <= threshold 

def shift_time(date_str, shift):
    new_date = datetime.strptime(date_str, d_format) + pd.Timedelta("{} hour".format(str(shift)))
    new_str = datetime.strftime(new_date, d_format)
    return new_str

def fill_na_values(i_df):
    counts = i_df["counts"]
    
    start_time = i_df["start_time"]
    # check whether continuous na on the bottom

    last_not_na_time = shift_time(start_time,-1)
    next_not_na_time = shift_time(start_time,counts)

    pollutant = i_df["pollutant"]

    last_df = aq_df[(aq_df["stationId"] == i_df["stationId"]) & \
                       (aq_df["time"] == last_not_na_time)]

    start_index = last_df.index.tolist()[0] + 1

    next_df = aq_df[(aq_df["stationId"] == i_df["stationId"]) & \
                       (aq_df["time"] == next_not_na_time)]

    last_value = last_df[pollutant].iloc[0]
    next_value = next_df[pollutant].iloc[0]

    interval = (next_value-last_value) / (counts+1)

    for i in range(counts):
#             aq_df.iloc[i+start_index][pollutant] = (last_value + (i+1)*interval)
        aq_df.set_value((i+start_index),pollutant,(last_value + (i+1)*interval))

    # what if next_not_na_time out of bound?
        
        
new_info_df = new_info_df.iloc[20100:20200]
new_info_df.apply(fill_na_values,axis=1)
    

77321


In [11]:
aq_df = aq_df.interpolate(limit=3)

In [12]:
aq_df[(aq_df["stationId"] == "aotizhongxin_aq") & \
      (aq_df["time"] >= "2017-03-08 00:00:00")]

,stationId,time,PM2.5,PM10,NO2,CO,O3,SO2
1570,aotizhongxin_aq,2017-03-08 00:00:00,7.000000,31.000000,24.000000,0.30,87.0,6.000000
1571,aotizhongxin_aq,2017-03-08 01:00:00,7.000000,36.000000,25.000000,0.40,86.0,5.000000
1572,aotizhongxin_aq,2017-03-08 02:00:00,12.000000,36.000000,16.000000,0.30,99.0,4.000000
1573,aotizhongxin_aq,2017-03-08 03:00:00,4.000000,36.000000,10.000000,0.30,111.0,3.000000
1574,aotizhongxin_aq,2017-03-08 04:00:00,8.000000,23.000000,9.000000,0.30,115.0,4.000000
1575,aotizhongxin_aq,2017-03-08 05:00:00,12.000000,29.000000,9.000000,0.30,119.0,4.000000
1576,aotizhongxin_aq,2017-03-08 06:00:00,12.000000,28.000000,8.000000,0.30,122.0,5.000000
1577,aotizhongxin_aq,2017-03-08 07:00:00,13.000000,33.000000,9.000000,0.30,122.0,5.000000
1578,aotizhongxin_aq,2017-03-08 08:00:00,17.000000,32.000000,9.000000,0.30,122.0,4.000000
1579,aotizhongxin_aq,2017-03-08 09:00:00,12.000000,27.000000,12.000000,0.30,119.0,4.000000


In [29]:
sr = pd.Series([0,1,np.nan,3,4,np.nan,np.nan,np.nan])
sr.interpolate(limit=1)

0    0.0
1    1.0
2    2.0
3    3.0
4    4.0
5    4.0
6    NaN
7    NaN
dtype: float64